In [1]:
import data_extraction
import pandas as pd
import yaml, boto3

pd.set_option('display.max_columns', None)

myObj = data_extraction.DataExtractor()

s3 = boto3.client('s3')

s3.download_file("data-handling-public",'date_details.json','date_details.json')



In [2]:
df = pd.read_json("date_details.json")

df is a dataframe that now contains the data from our table
These are the first & last few lines of the database then!

In [3]:
df.head(5)

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [4]:
df.tail(5)

,timestamp,month,year,day,time_period,date_uuid
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb
120160,21:17:12,3,2008,18,Evening,55c228c7-14ee-4d46-99a9-01dfc57d1adf


Make a string and put it in a new column

In [5]:
df['dateTime'] = df["year"] + "-" + df["month"]  + "-" +  df["day"] + " " + df["timestamp"]

In [6]:
df.head()

,timestamp,month,year,day,time_period,date_uuid,dateTime
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad,2012-9-19 22:00:06
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa,1997-2-10 22:44:06
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31,1994-4-15 10:05:37
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8,2001-11-6 17:29:27
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44,2015-12-31 22:40:33


Lets drop the old time columns

In [7]:
df = df.drop(columns=["timestamp","month","year","day","time_period"])

Convert datetime to datetime

In [8]:
df['dateTime']=pd.to_datetime(df['dateTime'],infer_datetime_format=True, errors='coerce')

Duplicated lines?

In [9]:
df.duplicated().sum()

14

14! ok. Let's just check on the critical columns only - not system generated ones

In [10]:
df.duplicated(subset=["date_uuid"],keep="first").sum()

14

Still 14
lets drop nas too

In [11]:
df.drop_duplicates(subset=["date_uuid"],keep="first",inplace=True)
df.dropna(inplace=True,axis="index",how="any") # drops the whole row if any columns have an nan


make sure we've not dropped everythign

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120123 entries, 0 to 120160
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date_uuid  120123 non-null  object        
 1   dateTime   120123 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 2.7+ MB


Now put the code into a local database maybe

In [13]:
import database_utils


myObject = database_utils.DatabaseConnector()


myObject.upload_to_db(df,"dim_date_times")

Ideas for future:
Misspellings: https://medium.com/geekculture/how-to-fix-misspelled-words-for-your-next-nlp-project-with-one-line-of-code-98888a5ff207
Common variants of companies - "ltd." vs "ltd" vs "limited"